# One hot encoding

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Let's load the data set

data = pd.read_csv("credit_approval_uci.csv")

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,1
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,1
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,1
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,1
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,1


In [3]:
# Let's separate into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=["target"], axis=1),  # predictors
    data["target"],  # target
    test_size=0.3,  # percentage of observations in test set
    random_state=0,  # seed to ensure reproducibility
)

X_train.shape, X_test.shape

((483, 15), (207, 15))

## One-hot encoding with pandas

In [4]:
# Let's inspect the unique values of A4

X_train["A4"].unique()

array(['u', 'y', 'Missing', 'l'], dtype=object)

In [5]:
# let's one-hot encode A4

dummies = pd.get_dummies(X_train["A4"], drop_first=False)

print(dummies.head())

     Missing      l      u      y
596    False  False   True  False
303    False  False   True  False
204    False  False  False   True
351    False  False  False   True
118    False  False   True  False


In [6]:
# let's one hot encode A4 into k-1 variables

dummies = pd.get_dummies(X_train["A4"], drop_first=True)

print(dummies.head())

         l      u      y
596  False   True  False
303  False   True  False
204  False  False   True
351  False  False   True
118  False   True  False


In [7]:
# Now let's encode all cateogrical variables simultaneously
# into k-1: train set

X_train_enc = pd.get_dummies(X_train, drop_first=True)

X_train_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_l,A4_u,...,A7_j,A7_n,A7_o,A7_v,A7_z,A9_t,A10_t,A12_t,A13_p,A13_s
596,46.08,3.000,2.375,8,396.0,4159,True,False,False,True,...,False,False,False,True,False,True,True,True,False,False
303,15.92,2.875,0.085,0,120.0,0,True,False,False,True,...,False,False,False,True,False,False,False,False,False,False
204,36.33,2.125,0.085,1,50.0,1187,False,True,False,False,...,False,False,False,True,False,True,True,False,False,False
351,22.17,0.585,0.000,0,100.0,0,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
118,57.83,7.040,14.000,6,360.0,1332,False,True,False,True,...,False,False,False,True,False,True,True,True,False,False


In [8]:
# and now in the test set.

X_test_enc = pd.get_dummies(X_test, drop_first=True)

X_test_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_l,A4_u,...,A7_j,A7_n,A7_o,A7_v,A7_z,A9_t,A10_t,A12_t,A13_p,A13_s
14,45.83,10.50,5.000,7,0.0,0,True,False,False,True,...,False,False,False,True,False,True,True,True,False,False
586,64.08,20.00,17.500,9,0.0,1000,False,True,False,True,...,False,False,False,False,False,True,True,True,False,False
140,31.25,3.75,0.625,9,181.0,0,True,False,False,True,...,False,False,False,False,False,True,True,True,False,False
492,39.25,9.50,6.500,14,240.0,4607,False,True,False,True,...,False,False,False,True,False,True,True,False,False,False
350,26.17,2.00,0.000,0,276.0,1,True,False,False,True,...,True,False,False,False,False,False,False,True,False,False


## One-hot encoding with Scikit-learn

In [9]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
# Make a list with the categorical variables

cat_vars = X_train.select_dtypes(include="O").columns.to_list()

cat_vars

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [11]:
# we create and train the encoder

encoder = OneHotEncoder(
    drop="first",  # to return k-1
    sparse_output=False,
)

In [12]:
# Indicate which variables to impute:
ct = ColumnTransformer([("encoder", encoder, cat_vars)],
                       remainder="passthrough").set_output(transform="pandas")

# Find the median value per variable:
ct.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('encoder',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10',
                                  'A12', 'A13'])])

In [13]:
ct.named_transformers_["encoder"].categories_

[array(['Missing', 'a', 'b'], dtype=object),
 array(['Missing', 'l', 'u', 'y'], dtype=object),
 array(['Missing', 'g', 'gg', 'p'], dtype=object),
 array(['Missing', 'aa', 'c', 'cc', 'd', 'e', 'ff', 'i', 'j', 'k', 'm',
        'q', 'r', 'w', 'x'], dtype=object),
 array(['Missing', 'bb', 'dd', 'ff', 'h', 'j', 'n', 'o', 'v', 'z'],
       dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['g', 'p', 's'], dtype=object)]

In [14]:
# Encode variables in the train and test sets

X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

In [15]:
# Scikit-learn returns a Numpy array

X_test_enc.head()

,encoder__A1_a,encoder__A1_b,encoder__A4_l,encoder__A4_u,encoder__A4_y,encoder__A5_g,encoder__A5_gg,encoder__A5_p,encoder__A6_aa,encoder__A6_c,...,encoder__A10_t,encoder__A12_t,encoder__A13_p,encoder__A13_s,remainder__A2,remainder__A3,remainder__A8,remainder__A11,remainder__A14,remainder__A15
14,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,45.83,10.50,5.000,7,0.0,0
586,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,64.08,20.00,17.500,9,0.0,1000
140,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,31.25,3.75,0.625,9,181.0,0
492,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,39.25,9.50,6.500,14,240.0,4607
350,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,26.17,2.00,0.000,0,276.0,1


In [16]:
# Obtain the binary variable names

ct.get_feature_names_out()

array(['encoder__A1_a', 'encoder__A1_b', 'encoder__A4_l', 'encoder__A4_u',
       'encoder__A4_y', 'encoder__A5_g', 'encoder__A5_gg',
       'encoder__A5_p', 'encoder__A6_aa', 'encoder__A6_c',
       'encoder__A6_cc', 'encoder__A6_d', 'encoder__A6_e',
       'encoder__A6_ff', 'encoder__A6_i', 'encoder__A6_j',
       'encoder__A6_k', 'encoder__A6_m', 'encoder__A6_q', 'encoder__A6_r',
       'encoder__A6_w', 'encoder__A6_x', 'encoder__A7_bb',
       'encoder__A7_dd', 'encoder__A7_ff', 'encoder__A7_h',
       'encoder__A7_j', 'encoder__A7_n', 'encoder__A7_o', 'encoder__A7_v',
       'encoder__A7_z', 'encoder__A9_t', 'encoder__A10_t',
       'encoder__A12_t', 'encoder__A13_p', 'encoder__A13_s',
       'remainder__A2', 'remainder__A3', 'remainder__A8',
       'remainder__A11', 'remainder__A14', 'remainder__A15'], dtype=object)

## One-hot encoding with Feature-engine

In [17]:
from feature_engine.encoding import OneHotEncoder

In [18]:
# let's create an encoder to return k-1 binary variables

ohe_enc = OneHotEncoder(drop_last=True)

In [19]:
# fit the encoder to the train set: it will learn the variables and
# categories to encode

ohe_enc.fit(X_train)

OneHotEncoder(drop_last=True)

In [20]:
# we can see which variables the encoder will encode

ohe_enc.variables_

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [21]:
# The categories that will be encoded

ohe_enc.encoder_dict_

{'A1': ['a', 'b'],
 'A4': ['u', 'y', 'Missing'],
 'A5': ['g', 'p', 'Missing'],
 'A6': ['c',
  'q',
  'w',
  'ff',
  'm',
  'i',
  'e',
  'cc',
  'x',
  'd',
  'k',
  'j',
  'Missing',
  'aa'],
 'A7': ['v', 'ff', 'h', 'dd', 'z', 'bb', 'j', 'Missing', 'n'],
 'A9': ['t'],
 'A10': ['t'],
 'A12': ['t'],
 'A13': ['g', 's']}

In [22]:
# let's transform train and test set

X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [23]:
# let's inspect the encoded train set

X_train_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_u,A4_y,...,A7_z,A7_bb,A7_j,A7_Missing,A7_n,A9_t,A10_t,A12_t,A13_g,A13_s
596,46.08,3.000,2.375,8,396.0,4159,1,0,1,0,...,0,0,0,0,0,1,1,1,1,0
303,15.92,2.875,0.085,0,120.0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
204,36.33,2.125,0.085,1,50.0,1187,0,1,0,1,...,0,0,0,0,0,1,1,0,1,0
351,22.17,0.585,0.000,0,100.0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
118,57.83,7.040,14.000,6,360.0,1332,0,1,1,0,...,0,0,0,0,0,1,1,1,1,0


In [24]:
# let's inspect the encoded test set

X_test_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_u,A4_y,...,A7_z,A7_bb,A7_j,A7_Missing,A7_n,A9_t,A10_t,A12_t,A13_g,A13_s
14,45.83,10.50,5.000,7,0.0,0,1,0,1,0,...,0,0,0,0,0,1,1,1,1,0
586,64.08,20.00,17.500,9,0.0,1000,0,1,1,0,...,0,0,0,0,0,1,1,1,1,0
140,31.25,3.75,0.625,9,181.0,0,1,0,1,0,...,0,0,0,0,0,1,1,1,1,0
492,39.25,9.50,6.500,14,240.0,4607,0,1,1,0,...,0,0,0,0,0,1,1,0,1,0
350,26.17,2.00,0.000,0,276.0,1,1,0,1,0,...,0,0,1,0,0,0,0,1,1,0


In [25]:
# The name of the variables in the transformed data

ohe_enc.get_feature_names_out()

['A2',
 'A3',
 'A8',
 'A11',
 'A14',
 'A15',
 'A1_a',
 'A1_b',
 'A4_u',
 'A4_y',
 'A4_Missing',
 'A5_g',
 'A5_p',
 'A5_Missing',
 'A6_c',
 'A6_q',
 'A6_w',
 'A6_ff',
 'A6_m',
 'A6_i',
 'A6_e',
 'A6_cc',
 'A6_x',
 'A6_d',
 'A6_k',
 'A6_j',
 'A6_Missing',
 'A6_aa',
 'A7_v',
 'A7_ff',
 'A7_h',
 'A7_dd',
 'A7_z',
 'A7_bb',
 'A7_j',
 'A7_Missing',
 'A7_n',
 'A9_t',
 'A10_t',
 'A12_t',
 'A13_g',
 'A13_s']

## One-hot encoding with Category Encoders

In [26]:
from category_encoders.one_hot import OneHotEncoder

In [27]:
# let's create the encoder to return k-1 binary variables
# Category Encoders always returns k-1 dummies

ohe_enc = OneHotEncoder(use_cat_names=True)

In [28]:
# fit the encoder to the train set: it will learn the variables and
# categories to encode

ohe_enc.fit(X_train)

OneHotEncoder(cols=['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13'],
              use_cat_names=True)

In [29]:
# The variables that will be encoded

ohe_enc.cols

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [30]:
# The names of the new variables

ohe_enc.get_feature_names_out()

array(['A1_a', 'A1_b', 'A1_Missing', 'A2', 'A3', 'A4_u', 'A4_y',
       'A4_Missing', 'A4_l', 'A5_g', 'A5_p', 'A5_Missing', 'A5_gg',
       'A6_c', 'A6_q', 'A6_w', 'A6_ff', 'A6_m', 'A6_i', 'A6_e', 'A6_cc',
       'A6_x', 'A6_d', 'A6_k', 'A6_j', 'A6_Missing', 'A6_aa', 'A6_r',
       'A7_v', 'A7_ff', 'A7_h', 'A7_dd', 'A7_z', 'A7_bb', 'A7_j',
       'A7_Missing', 'A7_n', 'A7_o', 'A8', 'A9_t', 'A9_f', 'A10_t',
       'A10_f', 'A11', 'A12_t', 'A12_f', 'A13_g', 'A13_s', 'A13_p', 'A14',
       'A15'], dtype=object)

In [31]:
# let's transform train and test set

X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [32]:
# let's inspect the encoded test set

X_test_enc.head()

,A1_a,A1_b,A1_Missing,A2,A3,A4_u,A4_y,A4_Missing,A4_l,A5_g,...,A10_t,A10_f,A11,A12_t,A12_f,A13_g,A13_s,A13_p,A14,A15
14,1,0,0,45.83,10.50,1,0,0,0,1,...,1,0,7,1,0,1,0,0,0.0,0
586,0,1,0,64.08,20.00,1,0,0,0,1,...,1,0,9,1,0,1,0,0,0.0,1000
140,1,0,0,31.25,3.75,1,0,0,0,1,...,1,0,9,1,0,1,0,0,181.0,0
492,0,1,0,39.25,9.50,1,0,0,0,1,...,1,0,14,0,1,1,0,0,240.0,4607
350,1,0,0,26.17,2.00,1,0,0,0,1,...,0,1,0,1,0,1,0,0,276.0,1
